In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Mars, Venus
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат) 1762 дней полета
date_launch = Time("2026-10-31", scale="utc").tdb
date_flyby_Mars = Time("2027-08-08", scale="utc").tdb
date_flyby_Venus = Time("2028-09-14", scale="utc").tdb
date_arrival = Time("2031-08-28", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Mars))
mar = Ephem.from_body(Mars, time_range(date_launch, end=date_flyby_Mars))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_mar = Orbit.from_ephem(Sun, mar, date_flyby_Mars)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_mar)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_mar, ss_ven)
ss_trans_2, ss_target_2 = ss_mar.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_ven, ss_jup)
ss_trans_3, ss_target_3 = ss_ven.apply_maneuver(man_lambert_3, intermediate=True)

In [6]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=186 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=324 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=185 * u.deg), color="crimson", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [9]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=186 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [12]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=324 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [46]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=185 * u.deg), color="crimson", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [47]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_mar, v_mar = Ephem.from_body(Mars, date_flyby_Mars).rv(date_flyby_Mars)
r_ven, v_ven = Ephem.from_body(Venus, date_flyby_Venus).rv(date_flyby_Venus)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [48]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

118149304.41809857 km
81710861.54458295 km
35433692.182054296 km

[-18.47406304  21.67151117   9.39488545] km / s


In [49]:
# Фазовые координаты Марса

print (f"{r_mar [0]}")
print (f"{r_mar [1]}")
print (f"{r_mar [2]}"'\n')

print (v_mar.to(u.km / u.s))

-157032138.95048374 km
-158416950.61186048 km
-68403311.61268112 km

[ 18.80401042 -12.7780619   -6.36833518] km / s


In [50]:
# Фазовые координаты Венеры

print (f"{r_ven [0]}")
print (f"{r_ven [1]}")
print (f"{r_ven [2]}"'\n')

print (v_ven.to(u.km / u.s))

58680408.7745513 km
83856342.20989549 km
34042873.16417938 km

[-29.54702782  16.49095898   9.28979942] km / s


In [51]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

-5466162.255899698 km
-722408500.8748132 km
-309501152.70643514 km

[12.91128957  0.59466289 -0.05942466] km / s


In [52]:
nh_tof_1 = date_flyby_Mars - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_mar), = iod.lambert(Sun.k, r_earth, r_mar, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=281.0000000068623>

In [53]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 3.1619866 km / s>

In [54]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [-1.85399815,  2.42359019,  0.82889123] km / s>

In [55]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end = (norm(vk_mar - v_mar)).to(u.km / u.s)
V_inf_end

<Quantity 3.01629227 km / s>

In [56]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end_vec = (vk_mar - v_mar).to(u.km / u.s)
V_inf_end_vec

<Quantity [-2.82017745,  0.19635223,  1.05169577] km / s>

In [57]:
nh_tof_2 = date_flyby_Venus - date_flyby_Mars # время полета Марс - Венера

(v0_mar, vk_ven), = iod.lambert(Sun.k, r_mar, r_ven, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=402.9999999922838>

In [58]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_0 = (norm(v0_mar - v_mar)).to(u.km / u.s)
V_inf_end_0

<Quantity 13.042237 km / s>

In [59]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_0_vec = (v0_mar - v_mar).to(u.km / u.s)
V_inf_end_0_vec

<Quantity [-11.28963233,  -6.51817625,   0.39689544] km / s>

In [60]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_2 = (norm(vk_ven - v_ven)).to(u.km / u.s)
V_inf_end_2

<Quantity 11.87060946 km / s>

In [61]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_2_vec = (vk_ven - v_ven).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [-9.45188781, -0.29768395, -7.1752749 ] km / s>

In [62]:
nh_tof_3 = date_arrival - date_flyby_Venus # время полета Венера - Юпитер

(V0_ven, Vk_Jup), = iod.lambert(Sun.k, r_ven, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=1078.0000000026882>

In [63]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_2_0 = (norm(V0_ven - v_ven)).to(u.km / u.s)
V_inf_end_2_0

<Quantity 14.13119728 km / s>

In [64]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_2_vec_0 = (V0_ven - v_ven).to(u.km / u.s)
V_inf_end_2_vec_0

<Quantity [-13.92518687,  -1.80785631,  -1.58479111] km / s>

In [65]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 6.75095747 km / s>

In [66]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k_vec = (Vk_Jup - v_jup).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [-6.66057617,  0.96715024,  0.52609158] km / s>

In [67]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

<TimeDelta object: scale='tdb' format='jd' value=1762.0000000018345>

In [68]:
scalar_EM = r_earth[0]*r_mar[0] + r_earth[1]*r_mar[1] +r_earth[2]*r_mar[2] 
scalar_MV = r_ven[0]*r_mar[0] + r_ven[1]*r_mar[1] +r_ven[2]*r_mar[2]
scalar_VJ = r_ven[0]*r_jup[0] + r_ven[1]*r_jup[1] +r_ven[2]*r_jup[2]

In [69]:
fi_EM = (scalar_EM/norm(r_earth)/norm(r_mar))
fi_MV = (scalar_MV/norm(r_mar)/norm(r_ven))
fi_VJ = (scalar_VJ/norm(r_ven)/norm(r_jup))

In [70]:
np.degrees(np.arccos(fi_EM))

<Quantity 169.31306145 deg>

In [35]:
np.degrees(np.arccos(fi_MV))

<Quantity 170.6022667 deg>

In [36]:
np.degrees(np.arccos(fi_VJ))

<Quantity 147.42356416 deg>

In [37]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 3.1619866 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 3.54966873 km / s>

In [38]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 6.7509575

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

12.918843495438685
17.983782441612995


<Quantity -5.06493895 km / s>

In [39]:
## параметры участка Землия-Марс 

from math import*

to_rad = pi/180

ist_an_1 = 356.4477161 * to_rad 
ist_an_2 = 186.4941861 * to_rad 

exs_EM = (norm(r_mar)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_mar)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.2249175628594252
Фокальный параметр 181171999.62039426 km
Большая полуось 190825460.87669995 km


In [40]:
## параметры участка Марс-Венера 

ist_an_1_MV = 136.5433673 * to_rad  
ist_an_2_MV = 327.4296561 * to_rad  

exs_MV = (norm(r_ven)/norm(r_mar) - 1) / (cos(ist_an_1_MV) - (norm(r_ven)/norm(r_mar) * cos(ist_an_2_MV)))
p_MV = norm(r_mar)*(1+cos(ist_an_1_MV)*exs_MV)
a_MV = p_MV/(1 - exs_MV*exs_MV)

print (f"Эксцентриситет {exs_MV}")
print (f"Фокальный параметр {p_MV}")
print (f"Большая полуось {a_MV}")

Эксцентриситет 0.48201835286830164
Фокальный параметр 151676712.94038853 km
Большая полуось 197583627.32335585 km


In [41]:
## параметры участка Венера-Юпитер

ist_an_1_VJ = 333.5703820 * to_rad   
ist_an_2_VJ = 184.5281640 * to_rad   

exs_VJ = (norm(r_jup)/norm(r_ven) - 1) / (cos(ist_an_1_VJ) - (norm(r_jup)/norm(r_ven) * cos(ist_an_2_VJ)))
p_VJ = norm(r_ven)*(1+cos(ist_an_1_VJ)*exs_VJ)
a_VJ = p_VJ/(1 - exs_VJ*exs_VJ)

print (f"Эксцентриситет {exs_VJ}")
print (f"Фокальный параметр {p_VJ}")
print (f"Большая полуось {a_VJ}")

Эксцентриситет 0.7704763022792089
Фокальный параметр 182280981.70063233 km
Большая полуось 448563269.6962056 km


In [42]:
## Вектор гиперболического избытка скорости после грав. маневра у Марса

r_pi_Mar = 3737.5965
muMar = 42828

V_inf_Mar = 3.0162923
V_inf_Mar_vec_t_0 = -2.8201775
V_inf_Mar_vec_t_1 = 0.19635223
V_inf_Mar_vec_t_2 = 1.0516958

betta_Mar = 2*asin(1/(1+((r_pi_Mar*V_inf_Mar*V_inf_Mar)/muMar)))
gamma_Mar = pi
Vxy_inf_Mar = sqrt (V_inf_Mar_vec_t_0**2 + V_inf_Mar_vec_t_1**2)

V_grav_X_Mar = V_inf_Mar_vec_t_0 * cos(betta_Mar) - (V_inf_Mar_vec_t_1*V_inf_Mar)/Vxy_inf_Mar * sin(betta_Mar)*cos(gamma_Mar) - (V_inf_Mar_vec_t_0*V_inf_Mar_vec_t_2)/Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Y_Mar = V_inf_Mar_vec_t_1 * cos(betta_Mar) + (V_inf_Mar_vec_t_0*V_inf_Mar)/Vxy_inf_Mar * sin(betta_Mar)*cos(gamma_Mar) - (V_inf_Mar_vec_t_1*V_inf_Mar_vec_t_2)/Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Z_Mar = V_inf_Mar_vec_t_2 * cos(betta_Mar) + Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Mar = sqrt (V_grav_X_Mar**2 + V_grav_Y_Mar**2 + V_grav_Z_Mar**2) 

print (f"betta_Mar      {betta_Mar*180/pi} deg")
print (f"V_grav_X_Mar   {V_grav_X_Mar * u.km / u.s}")
print (f"V_grav_Y_Mar   {V_grav_Y_Mar * u.km / u.s}")
print (f"V_grav_Z_Mar   {V_grav_Z_Mar * u.km / u.s}")
print (f"V_grav_Mar     {V_grav_Mar * u.km / u.s}")

betta_Mar      67.75494619365982 deg
V_grav_X_Mar   -0.8737239684163215 km / s
V_grav_Y_Mar   2.859389956074883 km / s
V_grav_Z_Mar   0.3981391291217089 km / s
V_grav_Mar     3.0162923034787066 km / s


In [43]:
## Вектор гиперболического избытка скорости после грав. маневра у Венеры

r_pi_ven = 6663.1676
muVen = 324859

V_inf_Ven = 11.870609
V_inf_end_vec_t_0 = -9.4518878
V_inf_end_vec_t_1 = -0.29768395
V_inf_end_vec_t_2 = -7.1752749

betta_ven = 2*asin(1/(1+((r_pi_ven*V_inf_Ven*V_inf_Ven)/muVen)))
gamma_ven = pi
Vxy_inf_ven = sqrt (V_inf_end_vec_t_0**2 + V_inf_end_vec_t_1**2)

V_grav_X_Ven = V_inf_end_vec_t_0 * cos(betta_ven) - (V_inf_end_vec_t_1*V_inf_Ven)/Vxy_inf_ven * sin(betta_ven)*cos(gamma_ven) - (V_inf_end_vec_t_0*V_inf_end_vec_t_2)/Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Y_Ven = V_inf_end_vec_t_1 * cos(betta_ven) + (V_inf_end_vec_t_0*V_inf_Ven)/Vxy_inf_ven * sin(betta_ven)*cos(gamma_ven) - (V_inf_end_vec_t_1*V_inf_end_vec_t_2)/Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Z_Ven = V_inf_end_vec_t_2 * cos(betta_ven) + Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Ven = sqrt (V_grav_X_Ven**2 + V_grav_Y_Ven**2 + V_grav_Z_Ven**2) 

print (f"betta_ven      {betta_ven*180/pi} deg")
print (f"V_grav_X_Ven   {V_grav_X_Ven * u.km / u.s}")
print (f"V_grav_Y_Ven   {V_grav_Y_Ven * u.km / u.s}")
print (f"V_grav_Z_Ven   {V_grav_Z_Ven * u.km / u.s}")
print (f"V_grav_Ven     {V_grav_Ven * u.km / u.s}")

betta_ven      29.790711557160765 deg
V_grav_X_Ven   -8.3884375022543 km / s
V_grav_Y_Ven   5.636446913258035 km / s
V_grav_Z_Ven   -6.227033680834143 km / s
V_grav_Ven     11.870609335558356 km / s
